In [9]:
import pandas as pd
import numpy as np

import os
import argparse
import pickle
import joblib
import pdb
import warnings

In [36]:
cohort_path = "/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/"
# tasks = ['hospital_mortality', 'sepsis', 'LOS_7', 'readmission_30','aki_lab_abnormal_label','aki_lab_aki1_label','aki_lab_aki2_label','aki_lab_aki3_label','hyperkalemia_lab_abnormal_label','hyperkalemia_lab_mild_label','hyperkalemia_lab_moderate_label','hyperkalemia_lab_severe_label','hypoglycemia_lab_abnormal_label','hypoglycemia_lab_mild_label','hypoglycemia_lab_moderate_label','hypoglycemia_lab_severe_label','hyponatremia_lab_abnormal_label','hyponatremia_lab_mild_label','hyponatremia_lab_moderate_label','hyponatremia_lab_severe_label','neutropenia_lab_mild_label','neutropenia_lab_moderate_label','neutropenia_lab_severe_label','anemia_lab_abnormal_label','anemia_lab_mild_label','anemia_lab_moderate_label','anemia_lab_severe_label','thrombocytopenia_lab_abnormal_label','thrombocytopenia_lab_mild_label','thrombocytopenia_lab_moderate_label','thrombocytopenia_lab_severe_label']
tasks = ['hospital_mortality', 'sepsis', 'LOS_7', 'readmission_30','aki_lab_aki3_label','hyperkalemia_lab_severe_label','hypoglycemia_lab_severe_label','hyponatremia_lab_severe_label','neutropenia_lab_severe_label','anemia_lab_severe_label','thrombocytopenia_lab_severe_label']


In [11]:
def read_file(filename, columns=None, **kwargs):
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [12]:
cohort = read_file(
    os.path.join(
        cohort_path,
        "cohort/cohort_split_no_nb.parquet"
    ),
    engine='pyarrow'
)

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort/cohort_split_no_nb.parquet


In [40]:
print(cohort[['person_id', 'fold_id', 'adult_at_admission']].groupby(['adult_at_admission','fold_id']).count())

                            person_id
adult_at_admission fold_id           
0                  0            20860
                   test          5581
                   val           3178
1                  0           185370
                   test         38035
                   val          21406


In [13]:
ad_cohort = cohort.query("adult_at_admission==1")
ped_cohort = cohort.query("adult_at_admission==0")

In [29]:
ped_cohort[["person_id", "fold_id"]].groupby(["fold_id"]).count()

,person_id
fold_id,
0,20860
test,5581
val,3178


In [15]:
ad_cohort.query("admission_year<=2019")[["person_id", "fold_id"]]#.groupby(["fold_id"]).count()

,person_id,fold_id
0,29936887,0
1,29936888,0
2,29936900,0
3,29936906,0
4,29936914,0
...,...,...
274191,43705036,0
274196,43705412,0
274216,43742869,0
274219,43743212,0


In [38]:
prev_df = pd.DataFrame()
for task in tasks:
    c_df = ped_cohort[["person_id",f"{task}"]]
    s = c_df.query(f"{task}==1")[task].sum()
    df = pd.DataFrame()
    df_dict = {"task":[task],
               "prevalence": [s/len(c_df)*100],
               "total_pos": [s],
               "total_pats": [len(c_df)]
              }
    df = pd.DataFrame(df_dict)
    prev_df = pd.concat((prev_df,df))
prev_df = prev_df.reset_index(drop=True)
with pd.option_context('display.max_rows', None,):
    print(prev_df)
        

                                 task  prevalence  total_pos  total_pats
0                  hospital_mortality    1.077011        319       29619
1                              sepsis    2.376853        704       29619
2                               LOS_7   19.892636       5892       29619
3                      readmission_30    5.645025       1672       29619
4                  aki_lab_aki3_label    0.975725        289       29619
5       hyperkalemia_lab_severe_label    1.796144        532       29619
6       hypoglycemia_lab_severe_label    1.718492        509       29619
7       hyponatremia_lab_severe_label    1.090516        323       29619
8        neutropenia_lab_severe_label    1.083764        321       29619
9             anemia_lab_severe_label    3.717209       1101       29619
10  thrombocytopenia_lab_severe_label    2.106756        624       29619


In [32]:
age_prev_df = pd.DataFrame()
for task in tasks:
    for aaa in [0, 1]:
        for fold in ["0", "val", "test"]:
            c_df = cohort[["person_id",f"{task}", f"{task}_fold_id", "adult_at_admission"]].query(f"{task}_fold_id==@fold and adult_at_admission==@aaa")
            s = c_df.query(f"{task}==1")[task].sum()
            df = pd.DataFrame()
            df_dict = {"task":[task],
                       "age_group": ['pediatric'] if aaa==0 else ['adult'],
                       "fold":["train"] if fold == "0" else [fold],
                       "prevalence": [s/len(c_df)*100],
                       "total_pos": [s],
                       "total_pats": [len(c_df)]
                      }
            df = pd.DataFrame(df_dict)
            age_prev_df = pd.concat((age_prev_df,df))
age_prev_df = age_prev_df.reset_index(drop=True)
with pd.option_context('display.max_rows', None,):
    print(prev_df)
        

                                 task  age_group   fold  prevalence  \
0                  hospital_mortality  pediatric  train    1.085850   
1                  hospital_mortality  pediatric    val    1.360329   
2                  hospital_mortality  pediatric   test    0.936599   
3                  hospital_mortality      adult  train    2.135311   
4                  hospital_mortality      adult    val    2.169691   
..                                ...        ...    ...         ...   
61  thrombocytopenia_lab_severe_label  pediatric    val    2.838915   
62  thrombocytopenia_lab_severe_label  pediatric   test    2.325581   
63  thrombocytopenia_lab_severe_label      adult  train    2.357426   
64  thrombocytopenia_lab_severe_label      adult    val    2.186002   
65  thrombocytopenia_lab_severe_label      adult   test    2.219475   

    total_pos  total_pats  
0         224       20629  
1          43        3161  
2          52        5552  
3        3907      182971  
4      

In [47]:
print([c for c in cohort.columns])

['person_id', 'admit_date', 'discharge_date', 'admit_date_midnight', 'discharge_date_midnight', 'hospital_mortality', 'death_date', 'month_mortality', 'LOS_days', 'LOS_7', 'readmission_30', 'readmission_window', 'icu_admission', 'icu_start_datetime', 'age_in_years', 'age_group_x', 'race_eth', 'gender_concept_name', 'race_eth_raw', 'race_eth_gender', 'race_eth_age_group', 'race_eth_gender_age_group', 'race_eth_raw_gender', 'race_eth_raw_age_group', 'race_eth_raw_gender_age_group', 'prediction_id', 'fold_id', 'adult_at_admission', 'sepsis', 'sepsis_index_date', 'admission_year', 'discharge_year', 'hospital_mortality_fold_id', 'sepsis_fold_id', 'LOS_7_fold_id', 'readmission_30_fold_id', 'icu_admission_fold_id', 'birth_datetime', 'age_days', 'pediatric_age_group', 'age_group_y', 'sex', 'race', 'hyperkalemia_lab_max_potassium', 'hyperkalemia_lab_mild_measurement', 'hyperkalemia_lab_mild_measurement_datetime', 'hyperkalemia_lab_mild_label', 'hyperkalemia_lab_moderate_measurement', 'hyperkale

In [41]:
ped_cohort[['person_id', 'gender_concept_name']].groupby(['gender_concept_name']).count()

,person_id
gender_concept_name,
FEMALE,14025
MALE,15592
No matching concept,2


In [48]:
# ped_cohort[['person_id', 'pediatric_age_group']].groupby(['pediatric_age_group']).count()
ped_cohort.query('pediatric_age_group=="term neonatal"')[['person_id', 'age_days']]

,person_id,age_days
511,30045358,5
512,30045395,1
639,30074819,5
659,30075744,3
1502,30285573,12
...,...,...
232266,80977196,4
232267,80977204,4
232276,85696704,3
232283,85697959,17


In [49]:
cohort_df = cohort.query('pediatric_age_group!="term neonatal"')
cohort_df[['person_id', 'pediatric_age_group']].groupby(['pediatric_age_group']).count()

,person_id
pediatric_age_group,
early adolescence,10656
early childhood,4017
infancy,4660
late adolescence,6201
middle childhood,5802
non-pediatric,237605
toddler,2302
unknown,4


In [21]:
cohort[['person_id', 'race_eth_raw']].groupby(['race_eth_raw']).count()

,person_id
race_eth_raw,
American Indian or Alaska Native,714
Asian,44324
Black or African American,10215
Hispanic or Latino,54334
Native Hawaiian or Other Pacific Islander,3191
Other,25158
White,136494


In [ ]:
# Check list of newborn ids and crosscheck with clmbr train ids